In [14]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,  chi2
from sklearn import svm
from scipy.sparse import csr_matrix


In [15]:
dom1 = []
dom2 = []
test_set = []
with open('data/domain1_train.json', 'r') as file:
    for line in file:
        dom1.append(json.loads(line))
        
with open('data/domain2_train.json', 'r') as file:
    for line in file:
        dom2.append(json.loads(line))
        
sam = pd.read_csv("data/sample.csv")

with open('data/test_set.json', 'r') as file:
    for line in file:
        test_set.append(json.loads(line))
        
dom1 = pd.DataFrame.from_dict(dom1)
dom2 = pd.DataFrame.from_dict(dom2)
dom2 = dom2[dom2['text'].apply(len) > 0]
test_set = pd.DataFrame.from_dict(test_set)
        

In [16]:
print(len(dom1))
print(len(dom2))
print(len(sam))
print(len(test_set))

19500
14899
1000
1000


In [17]:
dom1['label'].value_counts()
print(len(dom2.loc[dom2['label']==0]))
print(len(dom2.loc[dom2['label']==1]))

12750
2149


In [18]:
data = pd.concat([dom1, dom2], ignore_index=True)

In [19]:
from sklearn.model_selection import train_test_split
data['model'].fillna(8, inplace=True)


X = data[['text','model']] 
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=data[['model', 'label']], random_state=42)

In [20]:
y_train

9936     0
25220    0
24497    0
9566     1
18884    0
        ..
29451    0
4594     1
29348    0
26916    0
28576    0
Name: label, Length: 27519, dtype: int64

In [21]:
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

print(X_train['model'].value_counts(normalize=True))
print(X_test['model'].value_counts(normalize=True))

label
0    0.654094
1    0.345906
Name: proportion, dtype: float64
label
0    0.65407
1    0.34593
Name: proportion, dtype: float64
model
8.0    0.629347
0.0    0.068716
1.0    0.068534
3.0    0.068534
2.0    0.067989
6.0    0.051274
4.0    0.022930
5.0    0.022675
Name: proportion, dtype: float64
model
8.0    0.629360
0.0    0.068750
3.0    0.068605
1.0    0.068459
2.0    0.068023
6.0    0.051163
4.0    0.022965
5.0    0.022674
Name: proportion, dtype: float64


In [22]:
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np


X = X_train['text'].tolist()
X1 =X_test['text'].tolist()
max_length = max([len(seq) for seq in X])
word2vec = Word2Vec(X, vector_size=300, window=10, min_count=1, workers=4)

def get_text_vector(text, word2vec_model):
    vector_list = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv.index_to_key]
    if len(vector_list) == 0:
        return np.zeros(word2vec.vector_size)
    return np.mean(vector_list, axis=0)

X_train = np.array([get_text_vector(text, word2vec) for text in X])
X_test = np.array([get_text_vector(text, word2vec) for text in X1])

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks





smote = SMOTE(sampling_strategy=0.7)
tomek = TomekLinks(sampling_strategy='majority')


pipeline = Pipeline(steps=[('o', smote), ('u', tomek)])


X_train, y_train = pipeline.fit_resample(X_train, y_train)





In [23]:
print(y_train.value_counts(normalize=True))

label
0    0.586125
1    0.413875
Name: proportion, dtype: float64


In [24]:
model = svm.SVC()
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Test score: {score}")

Test score: 0.8638081395348837


In [25]:
X2 =test_set['text'].tolist()
test_set = np.array([get_text_vector(text, word2vec) for text in X2])
test_pre = model.predict(test_set)
result = pd.DataFrame({
    'class': test_pre,
})
result['id'] = result.index
result = result[['id'] + [col for col in result.columns if col != 'id']]
result

,id,class
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
995,995,0
996,996,1
997,997,0
998,998,1


In [26]:
result.to_csv('0.7_300.csv', index=False)
